In [ ]:
# Bloque 01: Instalación de paquetes e importación de bibliotecas
!pip install googletrans==4.0.0-rc1
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import whisper
from typing import Callable, TextIO
from googletrans import Translator
from google.colab import files, drive
import os
import shutil

In [ ]:
# Bloque 02: Montar Google Drive y mover archivos .mp4 a la carpeta /content
drive.mount('/content/drive')

ruta_origen = '/content/drive/MyDrive/videos'
ruta_destino = '/content/'

if not os.path.exists(ruta_destino):
    os.makedirs(ruta_destino)

for archivo in os.listdir(ruta_origen):
    if archivo.endswith('.mp4'):
        origen = os.path.join(ruta_origen, archivo)
        destino = os.path.join(ruta_destino, archivo)
        shutil.copy(origen, destino)
        print(f'El archivo {archivo} ha sido copiado a {ruta_destino}')

In [ ]:
# Bloque 03: Definir la función de transcripción
def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = '.'):
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"

In [ ]:
# Bloque 04: Transcripción de los archivos de video y guardado de los subtítulos en formato VTT, sin traducir
model = whisper.load_model("large")

# Crear la carpeta "Subtítulos" si no existe
if not os.path.exists("Subtítulos"):
    os.makedirs("Subtítulos")

# Guardar los archivos .vtt en la carpeta "Subtítulos"
for transcripcion in [ar for ar in os.listdir() if str.endswith(ar, "mp4")]:
    print(transcripcion)
    result = model.transcribe(transcripcion)
    st1 = "WEBVTT\n\n"
    for linea in result["segments"]:
        st1 += format_timestamp(linea["start"]) + " --> " + format_timestamp(linea["end"]) + "\n"
        st1 += linea["text"].strip() + "\n\n"
    with open(os.path.join("Subtítulos", transcripcion[:-3] + "vtt"), "w") as f:
        f.write(st1)
    print("Se guardó como " + os.path.join("Subtítulos", transcripcion[:-3] + "vtt"))

ruta_origen = '/content/Subtítulos'
ruta_destino = '/content/drive/MyDrive/SubtítulosEN'

# Comprobar si la ruta de destino existe, si no, crearla
if not os.path.exists(ruta_destino):
    os.makedirs(ruta_destino)

# Copiar todos los archivos de la ruta de origen a la ruta de destino
for archivo in os.listdir(ruta_origen):
    origen = os.path.join(ruta_origen, archivo)
    destino = os.path.join(ruta_destino, archivo)
    shutil.copy(origen, destino)
    print(f'El archivo {archivo} ha sido copiado a {ruta_destino}')

In [ ]:
# Bloque 05: Traducción individual de los subtítulos .vtt del inglés al español
if not os.path.exists("Subtítulos_ES"):
    os.makedirs("Subtítulos_ES")

translator = Translator()

for archivo_vtt in [ar for ar in os.listdir("Subtítulos") if str.endswith(ar, "vtt")]:
    print(archivo_vtt)
    with open(os.path.join("Subtítulos", archivo_vtt), "r") as f:
        contenido = f.read()
        lineas = contenido.split("\n")
        traducido = "WEBVTT\n\n"
        i = 2
        while i < len(lineas):
            if lineas[i] == "":
                traducido += "\n"
                i += 1
            else:
                traducido += lineas[i] + "\n"  # Copia la línea de tiempo
                i += 1
                texto_original = lineas[i]
                texto_traducido = translator.translate(texto_original.strip(), dest='es').text
                traducido += texto_traducido + "\n"
                i += 1
    with open(os.path.join("Subtítulos_ES", archivo_vtt), "w") as f:
        f.write(traducido)
    print("Se guardó como " + os.path.join("vtt_es", archivo_vtt))

In [ ]:
# Bloque 06: Guardado de los subtítulos .vtt traducidos al español
ruta_origen = '/content/Subtítulos_ES'
ruta_destino = '/content/drive/MyDrive/SubtítulosES'

if not os.path.exists(ruta_destino):
    os.makedirs(ruta_destino)
    
for archivo in os.listdir(ruta_origen):
    if archivo.endswith('.vtt'):
        origen = os.path.join(ruta_origen, archivo)
        destino = os.path.join(ruta_destino, archivo)
        shutil.copy(origen, destino)
        print(f'El archivo {archivo} ha sido copiado a {ruta_destino}')